In [11]:
# FUREY 
# https://www.furey.com.ar/

import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
import time
import credentials

def scrape_furey(login_url, products_url):
    session = requests.Session()

    payload = {'email': credentials.email_furey, 'password': credentials.password_furey}

    # Enviar la solicitud POST para iniciar sesión
    response = session.post(login_url, data=payload)

    # Verificar si el inicio de sesión fue exitoso
    if response.status_code == 200:
        print('Inicio de sesión exitoso')
    else:
        print('Inicio de sesión fallido')
        exit()

    # Crear un DataFrame vacío
    df = pd.DataFrame(columns=['name', 'price', 'brand', 'codigo de barra'])

    url = products_url

    while True:
        # Obtener el código HTML de la página web
        page = session.get(url)

        # Crear un objeto BeautifulSoup para analizar el código HTML
        soup = BeautifulSoup(page.content, 'html.parser')

        # Buscar todos los elementos con la clase 'product'
        products = soup.find_all('div', class_='product-box')

        # Iterar sobre cada producto
        for product in products:
            # Buscar el enlace del producto
            link = product.find('a')['href']

            # Acceder a la página del producto
            product_page = session.get(link)
            product_soup = BeautifulSoup(product_page.content, 'html.parser')

            # Buscar la información deseada (marca y dimensiones)
            # Buscar la fila de la marca
            try:
                brand_row = product_soup.find('td', class_='attrTitle', text='Marca')
            # Buscar el elemento siguiente a la fila de la marca, el cual contiene el valor de la marca
                brand = brand_row.find_next_sibling('td').text
            except AttributeError:
                brand = None

            # Codigo de Barra    
            try:
                codigo_row = product_soup.find('td', class_='attrTitle', text='Código de barras:')
            # Buscar el elemento siguiente a la fila de la marca, el cual contiene el valor de la marca
                codigo = codigo_row.find_next_sibling('td').text
            except AttributeError:
                codigo = None

            name = product_soup.find('h1', class_='tituloProducto').text
            price = product.find('meta', {'itemprop': 'price'})['content']

            # Agregar la información al DataFrame
            df.loc[len(df)] = [name, price, brand, codigo]

        # Construir la URL de la siguiente página
        next_url = None
        for a in soup.find_all('a', href=True, class_='pageResults'):
            if a.text.strip() == '»':
                next_url = a['href']
                break
        if next_url:
            url = next_url
            time.sleep(1)
        else:
            break


    # Guardar el DataFrame en un archivo CSV
    df.to_csv('../df/productos_furey.csv', index=False)
    print('Archivo guardado con exito')
    return True

scrape_furey('https://www.furey.com.ar/entrar.htm','https://www.furey.com.ar/listado-productos.htm?page=1')

Inicio de sesión exitoso
